Aprendizaje Automático - Práctica 2
Autores: RICARDO ANTONIO PAZOS VALERO - 100472303 / VICENTE ANTONIO BARBATO - 10043114

# 1. K-Means
---
Programamos el algoritmo K-means y comparamos nuestra implementación con la implementación de sklearn

In [1]:
# Importamos
import numpy as np
import pandas as pd
# import matplotlib.pylot as plt

# Cargamos los datos
data = pd.read_csv('data/stars_data.csv')

# Definimos nuestro conjunto de datos



   Temperature         L       R    A_M Color Spectral_Class
0         3068  0.002400  0.1700  16.12   Red              M
1         3042  0.000500  0.1542  16.60   Red              M
2         2600  0.000300  0.1020  18.70   Red              M
3         2800  0.000200  0.1600  16.65   Red              M
4         1939  0.000138  0.1030  20.06   Red              M
